In [1]:
from math import ceil, pi, e
from sympy import Symbol, diff, factorial, fraction, simplify, solve, N, symbols, linear_eq_to_matrix, factor_list, Matrix, BlockMatrix
from sympy.functions import sin, cos, tan
from termcolor import colored

In [2]:
def truncate(floatt):
  '''truncate floatt to n digits'''
  return round(round(floatt,10)*100000)/100000.0

def printt(string, color):
  print(colored(string,color))

def factoreq(eq):
  temp = abs(factor_list(eq)[0])
  return eq/temp

def getS(xs, fxs):
  arr = list()
  for i in range(len(xs)-1):
    arr.append([xs[i],xs[i+1]])
  print(arr)

  S = list()
  symbollist = list()
  print('\nS(x) =\t',end='')
  x = Symbol('x')
  for i in range(len(arr)):
    a,b,c,d = symbols(f'a{i} b{i} c{i} d{i}')
    symbollist.append([a,b,c,d,x])
    S.append(a+ b*(x-xs[i]) + c*(x-xs[i])**2 + d*(x-xs[i])**3)
    print(f'{S[-1]}\t= S{i}(x)\t{arr[i]}\n\t',end='')
  print()
  return S, symbollist

def partb(S, fxs, xs, symbollist):
  printt('============(PART B)============','red')
  length = len(S)
  diff=0
  for i in range(length):
    for j in range(length):
      print(f'S{i}(x{j+diff}) = f(x{j+diff})\t\t',end='')
    diff+=1
    print()

  print()
  diff=0
  equations = list()
  for i in range(length):
    x = symbollist[i][-1]
    for j in range(length):
      val = xs[j+diff]
      equations.append(S[i].subs(x,val) - fxs.get(val))
      print(f'S{i}({val}) = f({val}) => {S[i].subs(x,val)} = {fxs.get(val)}')
    diff+=1
  return equations
  
def partde(S, xs, equations, part):
  printt(f'\n============(PART {part})============', 'red')
  x = Symbol('x')
  deriv = list()
  if part=='D':
    derivtimes = 1
  elif part=='E':
    derivtimes = 2
  string = "'"*derivtimes
  for i in range(len(S)):
    deriv.append(diff(S[i],x,derivtimes))
    print(f"S{i}{string}(x) = {deriv[-1]}")

  print()
  for i in range(len(S)-1):
    print(f"S{i}{string}({xs[i+1]}) =  S{i+1}'({xs[i+1]})")
    print(f'{deriv[i].subs(x,xs[i+1])} = {deriv[i+1].subs(x,xs[i+1])}')
    equations.append(deriv[i].subs(x,xs[i+1]) - deriv[i+1].subs(x,xs[i+1]))
    print(f'{equations[-1]} = 0')
    print()
  return equations

def partf(S, fxs, xs, equations, typee, Sderivatives):
  printt(f'\n============(PART F)============', 'red')
  x = Symbol('x')
  if typee=='natural':
    equations.append(diff(S[0],x,2).subs(x,xs[0]))
    equations.append(diff(S[-1],x,2).subs(x,xs[-1]))

    print(f"S0''({xs[0]}) = 0 ",end='')
    printt(f'=> {equations[-2]} = 0','blue')
    print(f"S{len(S)-1}''({xs[-1]}) = 0",end='')
    printt(f'=> {equations[-1]} = 0','blue')
  else:
    equations.append(diff(S[0],x,1).subs(x,xs[0]) - Sderivatives[0])
    equations.append(diff(S[-1],x,1).subs(x,xs[-1]) - Sderivatives[-1])

    print(f"S0'({xs[0]}) = {Sderivatives[0]} ",end='')
    printt(f'=> {equations[-2]} = 0','blue')
    print(f"S{len(S)-1}'({xs[-1]}) = {Sderivatives[-1]} ",end='')
    printt(f'=> {equations[-1]} = 0','blue')

  return equations

In [3]:
def setvalues(equations,symbollist):
  printt(f'\n============(SOLVING)============', 'red')
  dictt = dict()
  neweqs = list()
  length = len(equations)
  printt(f'{length} equations','green')

  for i in range(length):
    eq = equations[i]
    temp = factoreq(eq)
    if eq==temp:
      print(f'{eq} = 0')
    else:
      print(f'{eq} = 0    ',end='')
      printt(f'=> {temp} = 0','blue')
      eq = temp
      equations[i] = temp
    symbs = list(eq.free_symbols)
    if len(symbs)==1:
      dictt.update({symbs[0]: solve(eq,symbs[0])[0]})
  printt('\n{Variable: value}','green')
  print(dictt)

  symbs = set()
  for eq in equations:
    for key, value in dictt.items(): 
      eq = eq.subs(key,value)
    if eq!=0:
      symbs.update(eq.free_symbols)
      neweqs.append(eq)

  printt(f'\n{len(neweqs)} equations left','green')
  [print(f'{i} = 0') for i in neweqs]
  symbs = sorted(list(symbs), key=lambda t:(str(t)[1],str(t)[0]))
  return neweqs, symbs, dictt

def makematrix(equations, symbs):
  printt(f'\n{symbs}','blue')
  return linear_eq_to_matrix(equations, symbs)

def printmatrix(mtrxs, symbs):
  length = len(symbs)
  [print(f'{mtrxs[0][i:i+length]} [{symbs[i//length]}] = [{mtrxs[1][i//length]}]') for i in range(0,len(mtrxs[0]),length)]

def geturl(mtrxs):
  url = 'https://www.emathhelp.net/calculators/linear-algebra/gauss-jordan-elimination-calculator/?i=%5B'
  length = len(mtrxs[1])
  temp = ''
  vals = list(mtrxs[1])
  vals.insert(0,0)
  for i in range(len(mtrxs[0])):
    if i%length==0:
      temp+=f'%2C{vals[i//length]}%5D%2C%5B{mtrxs[0][i]}'
    else:
      temp+=f'%2C{mtrxs[0][i]}'      
  
  temp = temp[10:] + f'%2C{vals[-1]}'
  url+=f'{temp}%5D%5D&reduced=1&steps=on'
  printt('\n============(GO TO THIS LINK FOR THE STEPS)============','red')
  print(url)

def getsolution(mtrxs, symbs, dictt):
  mtrx = list()
  length = len(mtrxs[1])
  for m in range(0,len(mtrxs[0]),length):
    mtrx.append(list(mtrxs[0][m:m+length]))
  for m in range(length):
    mtrx[m].append(mtrxs[1][m])
  reduced = Matrix(mtrx).rref()[0]
  ans = list(reduced.col(-1))
  for m in range(length):
    dictt.update({symbs[m]:ans[m]})
  return dictt

def printsol(S, dictt, xs, x):
  countt = 0
  for i in sorted(dictt.keys(), key=lambda t:str(t)[0]) : 
     countt+=1
     if countt%len(S):
       print()
     print(f'{i} = {dictt.get(i)}', end ='  \t') 
     
  print('\n\nS(x) =\t',end='')
  for i in range(len(S)):
    print(f'S{i}(x) = {S[i]}\n\t',end='')

  print('\nS(x) =\t',end='')
  for i in range(len(S)):
    print(f'S{i}(x) = {S[i].subs(dictt)}\t{xs[i:i+2]}\n\t',end='')

  if x!=None:
    temp = Symbol('x')
    print(f'\nS({x}) = ',end='')
    for i in range(len(S)):
      print(f'S{i}({x}) = {S[i].subs(dictt).subs(temp,x)}\t{xs[i:i+2]}\n\t ',end='')

In [4]:
def cubicsplineinterpolation(coords, typee, Sderivatives, x):  
  xs = [i[0] for i in coords]
  fxs = dict()
  [fxs.update({i[0]:i[1]}) for i in coords]

  S, symbollist = getS(xs, fxs)
  equations = partb(S, fxs, xs, symbollist)
  equations = partde(S, xs, equations,'D')
  equations = partde(S, xs, equations,'E')
  equations = partf(S, fxs, xs, equations, typee, Sderivatives)

  equations, symbs, dictt = setvalues(equations,symbollist)
  mtrxs = makematrix(equations, symbs)
  
  printmatrix(mtrxs, symbs)
  geturl(mtrxs)
  dictt = getsolution(mtrxs, symbs, dictt)
  printsol(S, dictt, xs, x)


In [5]:
coords = [(1,2),(2,3),(3,5)]
Sderivatives = [2, 1]     # S derivative values (for clamped)
x = None                    # x value to interpolate y
typee = 'natural'.lower() # natural or clamped

cubicsplineinterpolation(coords, typee, Sderivatives, x)

[[1, 2], [2, 3]]

S(x) =	a0 + b0*(x - 1) + c0*(x - 1)**2 + d0*(x - 1)**3	= S0(x)	[1, 2]
	a1 + b1*(x - 2) + c1*(x - 2)**2 + d1*(x - 2)**3	= S1(x)	[2, 3]
	
============(PART B)============
S0(x0) = f(x0)		S0(x1) = f(x1)		
S1(x1) = f(x1)		S1(x2) = f(x2)		

S0(1) = f(1) => a0 = 2
S0(2) = f(2) => a0 + b0 + c0 + d0 = 3
S1(2) = f(2) => a1 = 3
S1(3) = f(3) => a1 + b1 + c1 + d1 = 5

============(PART D)============
S0'(x) = b0 + c0*(2*x - 2) + 3*d0*(x - 1)**2
S1'(x) = b1 + c1*(2*x - 4) + 3*d1*(x - 2)**2

S0'(2) =  S1'(2)
b0 + 2*c0 + 3*d0 = b1
b0 - b1 + 2*c0 + 3*d0 = 0


============(PART E)============
S0''(x) = 2*(c0 + 3*d0*(x - 1))
S1''(x) = 2*(c1 + 3*d1*(x - 2))

S0''(2) =  S1'(2)
2*c0 + 6*d0 = 2*c1
2*c0 - 2*c1 + 6*d0 = 0


============(PART F)============
S0''(1) = 0 => 2*c0 = 0
S1''(3) = 0=> 2*c1 + 6*d1 = 0

============(SOLVING)============
8 equations
a0 - 2 = 0
a0 + b0 + c0 + d0 - 3 = 0
a1 - 3 = 0
a1 + b1 + c1 + d1 - 5 = 0
b0 - b1 + 2*c0 + 3*d0 = 0
2*c0 - 2*c1 + 6*d0 = 0    => c0 - c1 +